## Imports all libraries

In [ ]:
import requests
from urllib.request import urlopen

import json

import numpy as np
import pandas as pd

import plotly.express as px

from datetime import date, datetime

## Request the total cases in germany for every day.

In [ ]:
## Get data from the api

reqUrl = "https://api.corona-zahlen.org/germany/history/cases"

headersList = {
 "Accept": "*/*" 
}

payload = ""

response = requests.request("GET", reqUrl, data=payload,  headers=headersList)

data = response.json()

#### Prepare the format

In [ ]:
## Convert to a data frame

cases = pd.json_normalize(data['data'])

print(cases)

#### Start working with your data

In [ ]:
## Check the formats

print(cases.dtypes)

In [ ]:
## Convert date to date format

cases["date"] = pd.to_datetime(cases["date"])

print(cases.dtypes)

In [ ]:
## Sort data by date

cases.sort_values(by='date', ascending = True, inplace=True)

print(cases)

In [ ]:
## Sort data by most cases

cases.sort_values(by='cases', ascending = False, inplace=True)

print(cases)

In [ ]:
## Filter out current year's values

year = datetime.today().strftime('%Y')

cases_cy = cases[cases['date'].dt.year == int(year)]

print(cases_cy)

In [ ]:
## Group cases by months

cases_cy_by_month = cases_cy.groupby(cases_cy['date'].dt.strftime('%m %B'))['cases'].sum().sort_index()

print(cases_cy_by_month)

#### Visualize your data

In [ ]:
## Display cases over time

cases['month'] = cases['date'].dt.month

fig = px.bar(cases, x='date', y="cases")

fig.show()

### Request corona stats per state

In [ ]:
## Get data from the api

reqUrl = "https://api.corona-zahlen.org/states"

headersList = {
 "Accept": "*/*" 
}

payload = ""

response = requests.request("GET", reqUrl, data=payload,  headers=headersList)

data = response.json()


#### Convert to a Dataframe

In [ ]:
## Convert to a data frame

cases_by_state = pd.DataFrame.from_dict(data['data'], orient='index')

for index, row in cases_by_state.iterrows():
    delta = row["delta"]
    if not pd.isnull(delta):
        for k, v in delta.items():
            key = f'delta_{k}'
            if key not in cases_by_state:
                cases_by_state[key] = np.nan
            cases_by_state.loc[index, key] = v
cases_by_state.drop(columns=["delta"], inplace=True)

for index, row in cases_by_state.iterrows():
    hospitalization = row["hospitalization"]
    if not pd.isnull(hospitalization):
        for k, v in hospitalization.items():
            key = f'hospitalization_{k}'
            if key not in cases_by_state:
                cases_by_state[key] = np.nan
            cases_by_state.loc[index, key] = v
cases_by_state.drop(columns=["hospitalization"], inplace=True)

cases_by_state

#### Start working with your data

In [ ]:
## Check the formats

print(cases_by_state.dtypes)

In [ ]:
## Convert date to date format

cases_by_state["hospitalization_lastUpdate"] = pd.to_datetime(cases_by_state["hospitalization_lastUpdate"])
cases_by_state["hospitalization_date"] = pd.to_datetime(cases_by_state["hospitalization_date"])

print(cases_by_state.dtypes)

#### Visualize your data

In [ ]:
# Plot cases by state

with urlopen('https://raw.githubusercontent.com/isellsoap/deutschlandGeoJSON/main/2_bundeslaender/1_sehr_hoch.geo.json') as response:
    counties = json.load(response)

fig = px.choropleth(cases_by_state, geojson=counties, locations='name', featureidkey="properties.name", color='cases',
                           color_continuous_scale="OrRd",
                           projection="mercator",
                           range_color=(0, 5000000),
                           labels={'name':'State', 'cases':'Cases'}
                          )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(title_text = "Today's cases in Germany by state", margin={"r":0,"t":50,"l":0,"b":50})
fig.show()

In [ ]:
last_update = cases_by_state['hospitalization_lastUpdate'].iloc[1].strftime("%b %d %Y %H:%M")

fig = px.choropleth(cases_by_state, geojson=counties, locations='name', featureidkey="properties.name", color='hospitalization_cases7Days',
                           color_continuous_scale="OrRd",
                           projection="mercator",
                           range_color=(0, 1000),
                           labels={'name':'State', 'hospitalization_cases7Days':'Cases7Days'}
                          )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(title_text = f"Hospitalization cases of the last 7 days in Germany by state (Last Updated: {last_update})", margin={"r":0,"t":50,"l":0,"b":50})
fig.show()

### Request hospitalization stats per state

In [ ]:
## Get data from the api

reqUrl = "https://api.corona-zahlen.org/states/history/hospitalization"

headersList = {
 "Accept": "*/*" 
}

payload = ""

response = requests.request("GET", reqUrl, data=payload,  headers=headersList)

data = response.json()

hospitalization_by_state = pd.DataFrame.from_dict(data['data'], orient='index')

pd.DataFrame(hospitalization_by_state)

### Request recovered stats per state

In [ ]:
## Get data from the api

reqUrl = "https://api.corona-zahlen.org/states/history/recovered"

headersList = {
 "Accept": "*/*" 
}

payload = ""

response = requests.request("GET", reqUrl, data=payload,  headers=headersList)

data = response.json()

recovered_by_state = pd.DataFrame.from_dict(data['data'], orient='index')

pd.DataFrame(recovered_by_state)